*******************Create APSIM simulation file***************

Stuff you need to do at the beginning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
from lxml import etree as ET
from io import StringIO, BytesIO
%matplotlib inline
import os
import nbconvert
os.getcwd()

'D:\\GitHubRepos'

Get the base file and link to the code

In [2]:
ThisOne = open("BaseFile.apsimx", "r")
parser = ET.XMLParser(strip_cdata=False)
Base = ET.parse(ThisOne, parser)
BaseRoot = Base.getroot()
print(BaseRoot.tag)

IOError: [Errno 2] No such file or directory: 'BaseFile.apsimx'

In [3]:
ThisOne = open("PatchLib.xml", "r")
Patch = ET.parse(ThisOne, parser)
RawMasterPatch = Patch.getroot()
print(RawMasterPatch.tag)

Zone


Get the soils library and link to the code

In [4]:
ThisOne = open("SoilLib.xml", "r")
Soil = ET.parse(ThisOne, parser)
SoilRoot = Soil.getroot()
print(SoilRoot.tag)

Soil_Lib


Get the management library and link to the code

In [5]:
ThisOne = open("ManLib.xml", "r")
Manage = ET.parse(ThisOne, parser)
ManageRoot = Manage.getroot()
print(ManageRoot.tag)

Manager_Lib


Get the master table that tells python how to put simulation together

In [6]:
Table = pd.read_csv('Table.csv',  header=0) 
Table.Field = Table.Field.astype(object)
Table.Area = Table.Area.astype(object)
Table.Slice = Table.Slice.astype(object)
Table.PN = Table.PN.astype(object)
Table.Irrigator_ = Table.Irrigator_.astype(object)
Table.VRI = Table.VRI.astype(object)
Table = Table.applymap(str)
Table.head(3)

,Area,Zone,Patch,Landscape_,Irrigator_,PN,Slice,Field,Management,VRI
0,5,Soil_01,Patch_0001,Neutral,1,1,1,1,Manager_Field_01,15
1,5,Soil_01,Patch_0002,Neutral,2,2,1,1,Manager_Field_01,10
2,5,Soil_01,Patch_0003,Neutral,3,3,1,1,Manager_Field_01,5


In [15]:
Scenerios = pd.read_csv('Scenerios.csv',  header=0)
Scenerios.On_off = Scenerios.On_off.astype(str)
Scenerios.Start_irrig = Scenerios.Start_irrig.astype(str)
Scenerios.Finish_irrig = Scenerios.Finish_irrig.astype(str)
Scenerios.Irrig_eff = Scenerios.Irrig_eff.astype(str)
Scenerios.Nozzle_diameter = Scenerios.Nozzle_diameter.astype(str)
Scenerios.Pivot_Length = Scenerios.Pivot_Length.astype(str)
Scenerios.Rev_Time = Scenerios.Rev_Time.astype(str)
Scenerios.Flow_rate = Scenerios.Flow_rate.astype(str)
Scenerios.Rain_pause = Scenerios.Rain_pause.astype(str)
Scenerios.Days_pause = Scenerios.Days_pause.astype(str)
Scenerios.Crop_in = Scenerios.Crop_in.astype(str)
Scenerios.Application = Scenerios.Application.astype(str)
Scenerios.App_freq = Scenerios.App_freq.astype(str)
Scenerios.Resp_irrig = Scenerios.Resp_irrig.astype(str)
Scenerios.VRI = Scenerios.VRI.astype(str)
Scenerios.Soil_depth = Scenerios.Soil_depth.astype(str)
Scenerios.Irrig_paw = Scenerios.Irrig_paw.astype(str)
Scenerios.Irrig_method = Scenerios.Irrig_method.astype(str)

,Scenerios_no,On_off,Start_irrig,Finish_irrig,Irrig_eff,Nozzle_diameter,Pivot_Length,Rev_Time,Flow_rate,Rain_pause,Days_pause,Crop_in,Application,App_freq,Resp_irrig,VRI,Soil_depth,Irrig_paw,Irrig_method
0,1,True,10/01/1998,4/01/1999,100,10,600,24,15,100,3,True,15,3,False,False,600,50,Wettest
1,2,True,10/01/1998,4/01/1999,100,10,600,24,15,100,3,True,15,3,True,False,600,50,Wettest


Now for the magic! ;-) ....

Find the Simulation level

In [16]:
Sim = BaseRoot.getiterator('Simulation')

Using imported MasterPatch structure, change all the identifying factors and soil then insert new patches into the basefile

In [17]:
for y in range(0,Scenerios.Scenerios_no.count()):
    
    ThisOne = open("BaseFile.apsimx", "r")
    parser = ET.XMLParser(strip_cdata=False)
    Base = ET.parse(ThisOne, parser)
    BaseRoot = Base.getroot()
    
    Sim = BaseRoot.getiterator('Simulation')
    for SimLevel in Sim:
        a =SimLevel.tag
    
    for x in range(1,len(Scenerios.columns)):
        for temp in SimLevel.iter(tag=Scenerios.columns[x]):
            temp.text = Scenerios.iloc[y][x]

    for x in range(0,Table.Patch.count()):
    
        MasterPatch = copy.deepcopy(RawMasterPatch)
    
        MasterPatch[0].text = Table.Patch[x]
        #print(MasterPatch[0].text)
            
        MasterPatch[2][0].text = 'Report' + str(Table.PN[x])
        #print(MasterPatch[2][0].text)
            
        for temp in MasterPatch.iter(tag='This_field_no'):
            temp.text = Table.Field[x]
        
        for temp in MasterPatch.iter(tag='This_patch_no'):
            temp.text = Table.PN[x]
        
        for temp in MasterPatch.iter(tag='This_slice_no'):
            temp.text = Table.Slice[x]
        
        for temp in MasterPatch.iter(tag='Soil_type'):
            temp.text = Table.Zone[x]
        
        for temp in MasterPatch.iter(tag='Patch_area'):
            temp.text = Table.Area[x]
        
        for temp in MasterPatch.iter(tag='Landscape_position'):
            temp.text = Table.Landscape_[x]
    
        for temp in MasterPatch.iter(tag='Irrigator_position'):
            temp.text = Table.Irrigator_[x]
        
        for InsertSoil in SoilRoot.iter(tag=Table.Zone[x]):
            a =InsertSoil.tag
        CopyInsertSoil = copy.deepcopy(InsertSoil[0])
        MasterPatch.insert(4, CopyInsertSoil)
    
        CopyMasterPatch = copy.deepcopy(MasterPatch)
    
        Position = 9 + x
        SimLevel.insert(Position, CopyMasterPatch)
        
    ManageList = set(Table.Management)
    ManageList = list(ManageList)
    for x in range(0,len(ManageList)):
        for InsertManage in ManageRoot.iter(tag=ManageList[x]):
            a= InsertManage.tag
        CopyInsertManage = copy.deepcopy(InsertManage[0])
        Position = 8 + x
        SimLevel.insert(Position, CopyInsertManage)
        
    Base.write("Sim"+str(y+1)+".apsimx", pretty_print=True)
        
        
    #for zone in SimLevel.findall('Zone'):
        #SimLevel.remove(zone)



Run APSIM here...